In [ ]:
### Distance Matrix to MDS

### Get data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
### Import distance matrix and remove Row labels
df = pd.read_table("Data/ssRNA_complete_genomes_100_subsample_1000.dist")
df = df.drop("#query", axis=1)
### Get filenames from headers
INDEX = list(df)
print(len(INDEX))

### Get metadata

In [ ]:
import pandas as pd
df_Accessions_100 = pd.read_csv("Data/df_ssRNA_complete_100_subsample.tsv", sep="\t")
len(df_Accessions_100)

In [ ]:
Acc_to_species = dict(zip(df_Accessions_100["accs"], df_Accessions_100["species"]))

In [ ]:
import pickle
with open('Data/Accessions_100.pickle', 'rb') as handle:
    Accessions = pickle.load(handle)
len(Accessions)

In [ ]:
Sub_positions = []
Sub_accessions = []
for Acc in INDEX:
    if Acc in Accessions:
        Sub_positions.append(int(INDEX.index(Acc)))
        Sub_accessions.append(Acc)
print(len(Sub_positions))

In [ ]:
Sub_species = []
for Acc in Sub_accessions:
    Sub_species.append(Acc_to_species[Acc])

In [ ]:
Submatrix = df.iloc[Sub_positions, Sub_positions]

### Run MDS

In [ ]:
from sklearn import manifold

In [ ]:
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", random_state=2)
results = mds.fit(Submatrix)


In [ ]:
### Create plot
coords = results.embedding_
Plot = pd.DataFrame(dict(x=coords[:, 0], y=coords[:, 1], label=Sub_accessions))
Plot["Species"] = Sub_species
Plot.head()

### Plot MDS

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import pylab
from matplotlib import cm

In [ ]:
### Plot parameters
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

### Plot
groups = Plot.groupby('Species')
fig, ax = plt.subplots()
ax.margins(0.2) # Optional, just adds 5% padding to the autoscaling

colors = cm.jet(np.linspace(0, 1, len(groups)))
alphas = np.linspace(.2, .8, len(groups))
for group, color, alpha in zip(groups, colors, alphas):
    ax.plot(group[1].x, 
            group[1].y, 
            marker='o', 
            linestyle='', 
            ms=10, 
            label=group[0], 
            c=color, 
            markeredgecolor='k', 
            alpha=(float(1)/float(len(group[1])**(.2)))
           )

lgd = pylab.legend()
ax.legend_.remove()
plt.show()

### Run DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
def Run_DBSCAN(
                Distances, 
                Epsilon, 
                Min_group, 
                Labels_true
                ):

    db = DBSCAN(eps=float(Epsilon), min_samples= int(Min_group), metric='precomputed').fit(Distances)

    ### Parse Results from DBscan and Report ###
    Point_labels = db.labels_
    cluster_labels = np.unique(db.labels_)

    n_clusters_ = len(cluster_labels) - (1 if -1 in cluster_labels else 0)
    n_outliers_ = np.count_nonzero(Point_labels == -1)

    ### Calculate scores
    Homogeneity_score              =    metrics.homogeneity_score(Labels_true, Point_labels)
    Completeness_score             =    metrics.completeness_score(Labels_true, Point_labels)

    return [
            n_clusters_ , 
            n_outliers_, 
            Homogeneity_score, 
            Completeness_score, 
            ], Point_labels

In [ ]:
def Run_Multi_DBSCAN(
                        Labels_true, 
                        Distances, 
                        EPS_low        = 0.01, 
                        EPS_high       = 0.1, 
                        EPS_incr       = 0.001, 
                        Min_group_low  = 5, 
                        Min_group_high = 10, 
                        Min_group_incr = 1
                    ):

    All_Point_labels = []
    Stats=[["Epsilon", "Min_group", "Clusters", "Outliers", "Homogeneity_score", "Completeness_score"]]

    for Min_group in range(int(Min_group_low), int(Min_group_high)+int(Min_group_incr), int(Min_group_incr)):
        for Epsilon in range(int(EPS_low * 1000), int(EPS_high * 1000) + int(EPS_incr * 1000), int(EPS_incr * 1000)):
            Stat, Point_labels = Run_DBSCAN(Distances, Epsilon * 0.001, Min_group, Labels_true)
            Stats.append([str(Epsilon * 0.001), str(Min_group)] + Stat)
            All_Point_labels.append(Point_labels)
#             print(str(Epsilon * 0.001), str(Min_group))
    return Stats, All_Point_labels

In [ ]:
Multi_DBSCAN = Run_Multi_DBSCAN(labels_true,Distances, EPS_high = 0.4, EPS_low = 0.01, EPS_incr=0.01)

In [ ]:
i = 1
for Point_labels in Multi_DBSCAN[1]:
    silhouette_avg = silhouette_score(Distances, Point_labels)
    print("\t".join(Multi_DBSCAN[0][i][:2]), silhouette_avg)
    i+=1

In [ ]:
Distances = Submatrix.values
labels_true = Sub_species
db = DBSCAN(eps=float(.16), min_samples= int(9), metric='precomputed').fit(Distances)

In [ ]:
Point_labels = db.labels_
cluster_labels = np.unique(db.labels_)
n_clusters_ = len(set(cluster_labels))
n_outliers_ = np.count_nonzero(cluster_labels == -1)

silhouette_avg        =    silhouette_score(Distances, Point_labels)
Homogeneity_score     =    metrics.homogeneity_score(labels_true, Point_labels)
Completeness_score    =    metrics.completeness_score(labels_true, Point_labels)

print("Silhouette score:\t",silhouette_avg)
print("Homogeneity score:\t",Homogeneity_score)
print("Completeness score:\t",Completeness_score)

### Plot Silhouette

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import cm
plt.rcParams["figure.figsize"] = [16,12]

In [ ]:


silhouette_vals = silhouette_samples(Distances, Point_labels)
y_ax_lower, y_ax_upper = 0, 0
yticks=[]

Scores=[]
for i, c in enumerate(cluster_labels[1:]):
    Cluster_score = []
    c_silhouette_vals = silhouette_vals[Point_labels == c]
    c_silhouette_vals.sort()
    y_ax_upper += len(c_silhouette_vals)
    color = cm.jet(i / n_clusters_)
    ax = plt.barh(range(y_ax_lower,y_ax_upper),
            c_silhouette_vals,
            height = 1.0,
            edgecolor='none',
            color=color)
    yticks.append((y_ax_lower + y_ax_upper) / 2)
    y_ax_lower += len(c_silhouette_vals)
    Cluster_score.append(c_silhouette_vals)
    Scores.append([i, Cluster_score])
silhouette_avg = np.mean(silhouette_vals)
yline = plt.axvline(silhouette_avg,
            color = "black",
            linestyle="--")
yticks = plt.yticks(yticks, cluster_labels[1:] + 1)

In [ ]:
### Plot parameters
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

### Plot
groups = Plot.groupby('Species')
fig, ax = plt.subplots()
ax.margins(0.2) # Optional, just adds 5% padding to the autoscaling

colors = cm.tab10(np.linspace(0, 1, len(groups)))
alphas = np.linspace(.2, .8, len(groups))
for group, color, alpha in zip(groups, colors, alphas):
    ax.plot(group[1].x, 
            group[1].y, 
            marker='o', 
            linestyle='', 
            ms=10, 
            label=group[0], 
            c=color, 
            markeredgecolor='k', 
            alpha=(float(1)/float(len(group[1])**(.2)))
           )

lgd = pylab.legend()
# ax.legend_.remove()
plt.show()

In [ ]:

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))


# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.viridis(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0,0,0, 0]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14, alpha=.8)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14, alpha=.8)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()